# Uniqueness of a $[\![4,0,3]\!]_4$ stabiliser code

A $[\![4,0,3]\!]_4$ code is equivalent to a set of four solids (projective 3-spaces) in PG(7,2), each equipped with a symplectic polarity, such that every two solids span the entire space, and such that every codimension two space intersects an even number of solids in a hyperbolic line.

## Arcs of four solids in PG(7,2)

We first show that there are only three configurations of four solids in PG(7,2) such that every two solids span the entire space (i.e. an arc of four solids). Without loss of generality, the four solids are spanned by the four column blocks of a block matrix $$\begin{pmatrix}I&O&I&I\\O&I&I&A\end{pmatrix}$$ where every two column blocks have full rank, and $A$ is determined up to conjugation and permutation.

In [1]:
O = zero_matrix(GF(2),4)
I = identity_matrix(GF(2),4)

With the purpose of storing them as keys in a dictionary, the following funtion converts matrices into tuples:

In [2]:
def tuplify(M): return tuple(tuple(row) for row in M) # makes matrices hashable

The following function does row operations to bring a block matrix in "standard form", where the first column block is spanned by $\begin{pmatrix}I\\O\end{pmatrix}$, the second one by $\begin{pmatrix}O\\I\end{pmatrix}$ and the third one by $\begin{pmatrix}I\\I\end{pmatrix}$.

In [3]:
def standard_form(G):
    L1 = ~G[:,:8]
    G = L1*G # row-reduced form
    L2 = block_diagonal_matrix(I,G[:4,8:12]*~G[4:,8:12])
    G = L2*G # third column block is now spanned by [[I],[I]]
    return G, L2*L1 # we multiplied with L2*L1 on the left to bring into standard form

For any block $A$, the submatrices $\begin{pmatrix}I&I\\O&A\end{pmatrix}$ and $\begin{pmatrix}I&I\\I&A\end{pmatrix}$ should be invertible, i.e. $A$ and $A+I$ should be invertible.

In [4]:
GL = [M for M in MatrixSpace(GF(2),4,4) if M.is_invertible()]
blocks = [M for M in GL if (M+I).is_invertible()]

First, we determine all $A$'s up to conjugation.

In [5]:
conj_representatives_A = [] # A-blocks up to conjugation
get_conj_representative_A = {} # stores the conjugation representative of the A-block
for A in blocks:
    a = tuplify(A)
    if not a in get_conj_representative_A: # conjugacy class not yet considered
        conj_representatives_A.append(a)
        for X in GL:
            a_equiv = tuplify(~X*A*X)
            if not a_equiv in get_conj_representative_A:
                get_conj_representative_A[a_equiv] = a, X # conjugate with X to get A
len(conj_representatives_A)

5

Now also up to permutation.

In [6]:
representatives_A = [] # A-blocks up to equivalence
get_representative_A = {} # stores the representative of the A-block
for a, (a_conj, X) in get_conj_representative_A.items():
    if not a_conj in get_representative_A: # permutation class not yet considered
        representatives_A.append(a_conj)
        G = block_matrix(GF(2),[[1,0,1,1],[0,1,1,matrix(a_conj)]])
        columns = [G[:,4*i:4*i+4] for i in range(4)] # column blocks of G
        for pi in Permutations(4):
            Gnew, L = standard_form(block_matrix(GF(2), [[columns[pi(i+1)-1] for i in range(4)]])) # permuted and put in standard form
            a_equiv, Y = get_conj_representative_A[tuplify(Gnew[4:,12:]*~Gnew[:4,12:])] # conjugation representative of the new A-block
            if not a_equiv in get_representative_A:
                get_representative_A[a_equiv] = a_conj, ~L*~block_diagonal_matrix(Y,Y) # multiply with this on the left to get A
    a_perm, L = get_representative_A[a_conj]
    get_representative_A[a] = a_perm, L*block_diagonal_matrix(X,X) # multiply with this on the left to get A_perm
len(representatives_A) # number of configurations of four solids such that every two span the whole space

3

So there are three different arcs of four solids in PG(7,2).

In [7]:
for a in representatives_A:
    print(matrix(a), "\n")

[1 1 0 0]
[0 0 1 0]
[0 0 0 1]
[1 0 0 0] 

[0 1 1 0]
[1 0 0 0]
[0 0 0 1]
[0 1 0 0] 

[1 1 0 0]
[1 0 0 0]
[0 0 1 1]
[0 0 1 0] 



## $[\![4,0,3]\!]_4$ stabiliser code

For each of the three configurations of four solids, we check whether they can be equipped with a symplectic polarity such that every codimension two space intersects an even number of solids in a hyperbolic line. Suppose that this is the case. For each line $\ell$ in each of the four solids (so $4\cdot35=140$ lines), let $$x_\ell=\begin{cases}0\text{ if \(\ell\) is totally isotropic}\\1\text{ if \(\ell\) is hyperbolic}\end{cases}.$$
Then for each codimension two space $\pi$, the sum of the $x_\ell$'s for which $\ell$ is the intersection of $\pi$ with one of the four solids, equals zero modulo 2. So it suffices to check a system of homogeneous equations over $\mathbb{F}_2$, which we calculate as the nullspace of a matrix M. 

Given such a solution, one still has to check whether the lines $\ell$ for which $x_\ell=1$, really are hyperbolic lines of a symplectic polarity. W(3,2) contains 15 totally isotropic lines and 20 hyperbolic lines, so we first of all check that $|\{\ell\in\pi\mid x_\ell=1\}|=20$. Then, we choose a (potential) hyperbolic line $\ell_1$ and loop over all (potential) hyperbolic lines $\ell_2$ to check whether $\ell_2=\ell_1^\perp$ (we can fix $\ell_1$ by Witt's theorem). This is true if and only if every other (potential) hyperbolic line intersects exactly one of them. If we find such a line $\ell_2$, then these lines are indeed the hyperbolic lines of a symplectic polarity.

In [8]:
V = VectorSpace(GF(2),8)
codim2s = list(V.subspaces(6))
len(codim2s)

10795

In [9]:
# may take a few minutes
count = 0
for a in representatives_A:
    count += 1
    print("Calculating solutions for configuration nr. " + str(count) + ".")
    A = matrix(a)
    symmetries = [X for X in GL if X*A==A*X]
    solids = [V.subspace(list(block_matrix(GF(2),[x]))) for x in [[I,O],[O,I],[I,I],[I,A.transpose()]]]
    label = {} # stores the label of a line
    i = 0
    for solid in solids:
        for line in solid.subspaces(2):
            label[line] = i
            i += 1
    M = [] # 10795 x 140 matrix
    for pi in codim2s:
        row = [0]*140
        for solid in solids:
            intersection = pi.intersection(solid)
            if intersection.dimension() == 2:
                row[label[intersection]] = 1
        M.append(row)
    solutions = set()
    for v in matrix(GF(2), M).right_kernel():
        v = list(v)
        if tuple(v) in solutions:
            print("Solution found, but not new.")
            continue
        if not all([v[35*i:35*(i+1)].count(1)==20 for i in range(4)]):
            continue
        for solid in solids:
            symplectic_lines = [line for line in solid.subspaces(2) if v[label[line]]==1]
            l1 = symplectic_lines[0] # w.l.o.g. by Witt's theorem
            for l2 in symplectic_lines:
                symplectic = all(line.intersection(l1).dimension() != line.intersection(l2).dimension() for line in symplectic_lines) # if l2 = perp(l1)
                if symplectic:
                    break
            if not symplectic:
                break
        if symplectic:
            print("New solution found.")
            for X in symmetries:
                symmetric_solution = [0]*140
                for solid in solids:
                    for line in solid.subspaces(2):
                        if v[label[line]] == 1:
                            L = matrix(line.basis())*block_diagonal_matrix(X.transpose(),X.transpose())
                            newline = V.subspace(list(L))
                            symmetric_solution[label[newline]] = 1
                solutions.add(tuple(symmetric_solution))

Calculating solutions for configuration nr. 1.
Calculating solutions for configuration nr. 2.
Calculating solutions for configuration nr. 3.
New solution found.
Solution found, but not new.
Solution found, but not new.


So there are three solutions, all for the third configuration. However, they are equivalent up to a projective transformation that fixes the solids. In other words, there is a unique $[\![4,0,3]\!]_4$ code.